In [2]:
import re, string,os
from glob import glob as gb
import pandas as pd
from collections import Counter
from tqdm import tqdm
from datetime import datetime, timedelta, date
from collections import OrderedDict
import subprocess
from utils.functions import *
import seaborn as sns 
import matplotlib.pyplot as plt
from gensim.models import KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import strip_tags

In [3]:
output_dir = "~/Documents/GitHub/CrisisBureaucracy/results/w2v-models"

In [13]:
texts = []
for year in range(1957,1986):
    data = data_loader.subset(data_version="preprocessed_pm",start_date=year,end_date=year+1,words=[]).reset_index(drop=True)
    data = data.dropna()
    data.columns = [f"c{x}" for x in data.columns]
    data_b = data[data['c1'].astype(str).str.contains('bureauc')]
    data = data.sample(10000).reset_index(drop=True)
    data = data.append(data_b)
    data = data.drop_duplicates('c0')
    t = list(data.iloc[:,1])
    t = [[w for w in x.split(' ') if w != "" and w != " "] for x in t if len(x.split(' ')) > 10]
    #t = random.sample(t,2000)
    texts += t
    print(year,len(data_b),len(t))

1957 73 9063
1958 69 9099
1959 76 9126
1960 55 9006
1961 106 9174
1962 79 9213
1963 82 9114
1964 81 9053
1965 56 9136
1966 146 9225
1967 188 9199
1968 293 9381
1969 219 9287
1970 124 9088
1971 222 9368
1972 240 9321
1973 213 9251
1974 215 9322
1975 539 9660
1976 731 9878
1977 639 9767
1978 629 9750
1979 440 9529
1980 594 9784
1981 477 8409
1982 349 8131
1983 372 8376
1984 478 8122
1985 446 8569


In [16]:
texts = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [17]:
model = Doc2Vec(documents=texts,
                 vector_size=300,
                 min_count=25, window=15,
                 sample=1e-5,
                 negative=5,
                 hs=1,
                 epochs=75,
                 dm=0,
                 dbow_words=1,
                 workers=8)

In [18]:
model.save(f"/home/ruben/Documents/GitHub/CrisisBureaucracy/results/w2v-models/doc2vec-v300-mc25-s200y.model")